<a href="https://colab.research.google.com/github/barbaroja2000/llm/blob/main/Langchain_%26_Runpod_Meeting_Transcript_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Langchain Runpod Meeting Transcript Analyser

This colab shows how to run Llama2-7b-chat-hf model on runpod. & demo meeting transcript analyser app.

*   Participants
*   Meeting topic (metadata or parsed)
*   Meeting summary
*   Meeting date, time, location (metadata or parsed)
*   Meeting actions & deadlines
*   Decisions Made
*   Questions: Raised (and possibly unanswered)

Notes:

* Embedding model -  "sentence-transformers/all-mpnet-base-v2"
* FAISS for Vector store - swap out with pinecone for persistant vector store
* Model meta-llama/Llama-2-7b-chat-hf
* Chunk size for documents 1000 char with 100 overlap
* 512 max new tokens Llama-2-7b-chat-hf


In [1]:
#@title Load Keys
#@markdown Utitily to load keys from fs, replace with environ vars if not using
import os

!python -m pip install python-dotenv
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
import dotenv
dotenv.load_dotenv('/content/drive/MyDrive/keys/keys.env')

Mounted at /content/drive/


True

In [2]:
!wandb login ${WANDB_API_KEY}

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
# turn on wandb logging for langchain
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

In [5]:
# @title  Synthetic .vtt meeting transcript
synthetic_transcript_uri="https://gist.githubusercontent.com/barbaroja2000/277fd35e17ae6bc8610c29591f39c3a9/raw/5ecd4dc010e98c54f2d1537835a6acff4317443a/synthetic-transcript"

In [6]:
import requests

def fetch_text_file(url, save_path):
    """
    Fetch a text file from a URL and save it locally.

    Parameters:
    - url (str): The URL of the text file.
    - save_path (str): Local path where the file should be saved.
    """

    response = requests.get(url)

    # Ensure the request was successful
    response.raise_for_status()

    # Write the content to a local file
    with open(save_path, 'w', encoding=response.encoding) as file:
        file.write(response.text)


save_path = 'synthetic_transcript.vtt'
fetch_text_file(synthetic_transcript_uri, save_path)

In [7]:
!pip install -qU text-generation runpod langchain faiss-cpu huggingface_hub sentence_transformers wandb > /dev/null

In [8]:
!pip install --upgrade 'urllib3<2' #required for AttributeError: module 'urllib3.util' has no attribute 'PROTOCOL_TLS'

In [10]:
import runpod
import os
from IPython.display import display, Markdown

runpod.api_key = os.getenv("RUNPOD_API_KEY", "your_runpod_api_key")

if runpod.api_key == "your_runpod_api_key":
    display(
        Markdown(
            "It appears that you don't have a RunPod API key. You can obtain one at [runpod.io](https://runpod.io?ref=s7508tca)"
        )
    )
    raise AssertionError("Missing RunPod API key")

In [178]:
gpu_count = 1

pod = runpod.create_pod(
    name="Llama-2-7b-chat",
    image_name="ghcr.io/huggingface/text-generation-inference:0.9.4",
    gpu_type_id="NVIDIA RTX A4500",
    data_center_id="EU-RO-1",
    cloud_type="SECURE",
    docker_args=f"--model-id TheBloke/Llama-2-7b-chat-fp16",
    gpu_count=gpu_count,
    volume_in_gb=50,
    container_disk_in_gb=5,
    ports="80/http",
    volume_mount_path="/data",
)

{'data': {'podFindAndDeployOnDemand': {'id': '53tvbee3v9z421', 'imageName': 'ghcr.io/huggingface/text-generation-inference:0.9.4', 'env': [], 'machineId': 'b7yhdwws8mel', 'machine': {'podHostId': '53tvbee3v9z421-64410c16'}}}}


In [33]:
#previously running pod
#pod = {}
#pod["id"] = ""

In [23]:
inference_server_url = f'https://{pod["id"]}-80.proxy.runpod.net'

In [24]:
import time

def wait_for_200(url, timeout=60):
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            print("Received 200 status code!")
            return
        else:
            print(f"Status code: {response.status_code}. Retrying in {timeout} seconds...")
            time.sleep(timeout)

wait_for_200(inference_server_url)

Received 200 status code!


In [25]:
from langchain.llms import HuggingFaceTextGenInference

llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.03,
)

In [26]:
print(f"Docs (Swagger UI) URL: {inference_server_url}/docs")

Docs (Swagger UI) URL: https://53tvbee3v9z421-80.proxy.runpod.net/docs


In [17]:
from langchain.document_loaders import TextLoader

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import HuggingFaceTextGenInference
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from langchain.document_loaders import DirectoryLoader
from langchain.chains.question_answering import load_qa_chain
import os , requests
from typing import List, Dict
import glob
from langchain.chains.summarize import load_summarize_chain
import re

In [15]:
HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [27]:
# @title  SummarizeNQA Class

class SummarizeNQA:
    def __init__(self, key: str,  llm: HuggingFaceTextGenInference, dir: str) -> None:
        if not key:
            raise ValueError("API key must be provided.")
        if not dir  or not os.path.isdir(dir):
            raise ValueError("Directory must be provided.")

        self.dir = dir
        self.llm = llm

        self.db = None
        self.docs = None

    def load(self, chunk_size: int = 1000, chunk_overlap: int = 100) -> None:

        documents = []
        if not glob.glob(f"{self.dir}*.*"):
            raise ValueError("Directory must contain at least one file.")

        if  glob.glob(f"{self.dir}*.vtt"):
          loader = DirectoryLoader(
              "", glob=f"{self.dir}*.vtt", loader_cls=TextLoader
          )
          documents = [*loader.load(), *documents]

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )
        self.docs  = text_splitter.split_documents(documents)
        embeddings = HuggingFaceEmbeddings( model_name="sentence-transformers/all-mpnet-base-v2") #what other ones to use here
        self.db = FAISS.from_documents(self.docs, embeddings)

    def summarize(self, max_tokens=1000,chain_type='map_reduce' ):

      if not self.db:
         raise ValueError("Load first")

      map_prompt = """
                Write a  summary of the following:
                "{text}"
                 SUMMARY:
                """
      map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

      combine_prompt = """
      Write a  summary of the following text delimited by triple backquotes.
      Return your response in bullet points which covers the key points of the text.
      ```{text}```
      BULLET POINT  SUMMARY:
      """
      combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

      summary_chain = load_summarize_chain(llm=self.llm,
              chain_type=chain_type,
              map_prompt=map_prompt_template,
              combine_prompt=combine_prompt_template,
              verbose=False, return_intermediate_steps=False
          )
      response = summary_chain.run(self.docs)
      return  self._format(response)

    def qa(
        self,
        query: str,
        temperature: float = 0,
        count: float = 4,
        chain_type: str = "stuff", #map_reduce, #refine
        return_only_outputs: bool = True,
        return_intermediate_steps: bool = False,
    ) -> Dict:

        docs = self.db.similarity_search(query, k=count)

        question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question.

        {context}
        Question: {question}
        Relevant text, if any:"""

        QUESTION_PROMPT = PromptTemplate(
            template=question_prompt_template, input_variables=["context", "question"]
        )

        combine_prompt_template = """Given the following extracted parts of a long document and a question, create a final answer.
        If you don't know the answer, just say that you don't know. Don't try to make up an answer.

        QUESTION: {question}
        =========
        {summaries}
        =========
        Answer :"""

        COMBINE_PROMPT = PromptTemplate(
              template=combine_prompt_template, input_variables=["summaries", "question"]
        )

        if chain_type == "stuff":
            chain = load_qa_chain(self.llm, chain_type=chain_type)
        else:
            chain =  load_qa_chain(self.llm, chain_type=chain_type, return_intermediate_steps=return_intermediate_steps, question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT)
        response =  chain(
            {"input_documents": docs, "question": query},
            return_only_outputs=return_only_outputs,
        )
        return self._format(response["output_text"])

    @staticmethod
    def _format(answer: str) -> str:
        return re.sub('\n{3,}', '\n\n', answer)


In [28]:
# @title  Set Up
summarize_and_qa = SummarizeNQA(os.environ.get("HUGGINGFACEHUB_API_TOKEN"),llm=llm, dir="./")

In [29]:
# @title Load the texts into documents and index
summarize_and_qa.load()

In [30]:
# @title Summarize
# @markdown ```Depending on the length of the text, you may have to reduce the max_token parameter```

summary = summarize_and_qa.summarize(max_tokens=1000)

wandb: Streaming LangChain activity to W&B at https://wandb.ai/bandulu/uncategorized/runs/58vieul0
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


In [31]:
print(summary)

• The meeting discussed the potential of AI in the company's services, specifically in predictive analytics and automation.
       • Participants discussed the challenges of differentiation in a crowded market and explored collaboration as a means to gain an edge.
       • The importance of technical capabilities to support AI operations, particularly generative models, was highlighted.
       • The need for a comprehensive evaluation of the company's infrastructure to support the development of AI products was discussed.
       • The possibility of hosting a workshop for clients to demonstrate the company's AI capabilities was raised.
       • The importance of continuous learning and collaboration with universities for staying ahead in the AI field was emphasized.
       • The integration of generative AI into the company's product lineup and the need for a clear message to communicate the company's capabilities in the AI domain were discussed.
       • Ethical implications of genera

In [167]:
# @title Meeting Participants
response = summarize_and_qa.qa("list the meeting attendees",chain_type="map_reduce")
print(response)



The answer is:

* AC Head of Architecture
* BD Managing Director
* AJ Principal Architect
* MM Enterprise Architect
* GC IT Director


In [168]:
# @title Meeting Topic
response = summarize_and_qa.qa("what was the topic of the meeting",chain_type="map_reduce")
print(response)

ethical implications of generative AI


In [169]:
# @title Actions and Deadlines
response = summarize_and_qa.qa("describe the meeting follow-on actions and any deadlines",chain_type="map_reduce")
print(response)

The meeting follow-on actions and deadlines are as follows:
* Coordinate with the sales team to arrange a workshop on AI capabilities and drive sales in the coming months.
* Gather feedback from clients to understand their specific needs in AI solutions.
* Draft a proposal for the team structure and responsibilities.
* Reconvene in a week to review progress.
* A report on infrastructure needs should be ready in about three weeks.
* Set up a dedicated team to ensure that AI solutions comply with data protection regulations.
* Take action on the idea of continuous learning.
* Collaborate with universities to provide training and academic insights.
* Consider setting aside a budget for internal research and development.




In [170]:
# @title Decisions Made
response = summarize_and_qa.qa("List the decisions made in the meeting",chain_type="map_reduce")
print(response)

The decisions made in the meeting are:

A) Setting up a dedicated AI support team
B) Drafting a proposal for the team structure and responsibilities
C) Needing a mix of technical and domain experts in the team
D) Conducting a workshop for clients
E) Gathering feedback from clients
F) Training is essential
G) Evaluating the company's infrastructure needs
H) Forming an ethics committee for AI
I) Implementing safeguards against misuse of generative AI
J) Using "Ethically Designed AI Solutions" as a selling point
K) Providing post-deployment support for AI solutions

Note: The above answer is based on the information provided in the extract and may not reflect the complete picture of the meeting.


In [171]:
# @title Questions raised
response = summarize_and_qa.qa("list all the questions raised in the meeting, and answers provided.",chain_type="map_reduce")
print(response)



The answers to the questions raised in the meeting are as follows:

* Set up a dedicated AI support team: The proposal for a dedicated AI support team was raised, but no answer was provided.
* Proposal for team structure and responsibilities: The proposal for a team structure and responsibilities was raised, and it was mentioned that the team will consist of a mix of technical and domain experts.
* Review progress in a week: The question of reviewing progress in a week was raised, but no answer was provided.
* Infrastructure needs for scalable AI: The need for infrastructure to support scalable AI was raised, and it was discussed that partnerships in the tech space and security concerns will be evaluated.
* Evaluation of infrastructure needs: The question of evaluating infrastructure needs was raised, but no answer was provided.
* Ethical implications of generative AI: The ethical implications of generative AI were discussed, and it was mentioned that a dedicated ethics committee for

In [32]:
# Stop the pod
runpod.stop_pod(pod["id"])

# Terminate the pod
runpod.terminate_pod(pod["id"])

{'data': {'podStop': {'id': '53tvbee3v9z421', 'desiredStatus': 'EXITED'}}}
{'data': {'podTerminate': None}}
